In [8]:
import os
import glob

import xarray as xr
import numpy as np

In [41]:
data_m = '/home/hannasv/Desktop/lagrings/ERA5_monthly/'

In [42]:
test = '2005-05'

# Data exploration we want to study :
* Correlation pixelwise to pressure
* Everything as a mean over time 
* Hvordan dele opp dataen slik at det kan leses inn best mulig. 
* Hugo minnet sette begrensninger for AR - models med invers matrise?
* Hvor mye data har jeg egentlig?

In [ ]:
# Hyperparameters useful stuff - 
num_epochs = 4
batch_size = 32
#learning_rate = 0.001

#DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
#MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

In [3]:
"%2.2d" %3

'03'

In [2]:
'2012_02' < '2012_01'

False

In [21]:
def get_files(start, stop):
    """ Get files in requested period.
    Insert test on start > stop. 
    
    Returns:
        List of files to be read into 
    
    """
    y, m = start.split('_')
    start_y = int(y)
    start_m = int(m)
    
    y, m = stop.split('_')
    stop_y = int(y)
    stop_m = int(m)

    years = np.arange(start_y, stop_y+1)
    months = np.arange(1, 13)  
    search_str = [start, stop]
    
    for y in years:
        for m in months:
            m = "%2.2d" %m
            tmp = '{}_{}'.format(y, m)
            if tmp > start and tmp < stop:
                search_str.append(tmp)   
    
    storage = '/home/hannasv/Desktop/lagrings/ERA5_monthly/'
    
    files = []
    for ss in search_str:
        tmp_files = glob.glob(os.path.join( storage, '*{}*.nc'.format(ss)  ))  
        files += tmp_files
    return files

In [23]:
# get_files('2012_03', '2014_01')

In [43]:
# Loader of AR 

def load_data_AR(start, stop, lat, lon, num_ts = 0, bias = True, validation_split = 0.0):
    """ Prepares data for 
    
    start, stop : str
        year_month. Limited to montly values.
        
    lat, lon : float
        coordinate information
        
    bias : bool 
        Default 1
        
    validation_split : float
        Number between 0 and 1. 
        Intended to make it similar to keras input data. Will it make it easier to train networks though...?
    """
    if validation_split > 0:
        raise NotImplementedError('Comming soon ... Make sure that you actually need this?')
        
    # Search for list of files in the range start stop.
    # Returns list of strings you can merge to a dataset 
    
    files = get_files(start, stop)
    datasets = [ xr.open_dataset(fil).sel(latitude = lat, longitude = lon) for fil in files]
    data = xr.merge(datasets,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 'equals') # fill_value = np.nan,
    print(data)
    n     = len(data.time.values)
    n_lat = len(data.latitude.values)
    n_lon = len(data.longitude.values)
    
    q   = data.q.values
    t2m = data.t2m.values
    r   = data.r.values
    sp  = data.sp.values
    
    tcc = data.tcc.values
    
    if bias:
        num_vars = 4+1+num_ts
    else:
        num_vars = 4 + num_ts
        
    num_samples = n - num_ts
    X = np.zeros((num_samples, n_lat, n_lon, num_vars))
    
    X[:, :, :, 0] = 1
    X[:, :, :, 1] = t2m
    X[:, :, :, 2] = r
    X[:, :, :, 3] = sp
    X[:, :, :, 4] = q
    
    for i in range(1, num_ts+1):
        if num_ts - i != 0:
            X[:, :, :, 4+i] = tcc[i:num_ts-i]
        else:
            X[:, :, :, 4+i] = tcc[i:]
    # Now drop rows contaning nans.
    print(X.shape)
    no_nans =  X[~np.isnan(X.any(axis=1))]
    print(no_nans.shape)
    return no_nans
    

In [44]:
start='2012_02'
stop='2012_04'

lat=30
lon = 15

In [45]:
xr.merge?

Signature: xr.merge(objects, compat='no_conflicts', join='outer')
Docstring:
Merge any number of xarray objects into a single Dataset as variables.

Parameters
----------
objects : Iterable[Union[xarray.Dataset, xarray.DataArray, dict]]
    Merge together all variables from these objects. If any of them are
    DataArray objects, they must have a name.
compat : {'identical', 'equals', 'broadcast_equals',
          'no_conflicts'}, optional
    String indicating how to compare variables of the same name for
    potential conflicts:

    - 'broadcast_equals': all values must be equal when variables are
      broadcast against each other to ensure common dimensions.
    - 'equals': all values and dimensions must be the same.
    - 'identical': all values, dimensions and attributes must be the
      same.
    - 'no_conflicts': only values which are not null in both datasets
      must be equal. The returned dataset then contains the combination
      of all non-null values.
join : {'outer'

In [46]:
load_data_AR(start, stop, lat, lon, num_ts = 0, bias = True, validation_split = 0.0)

<xarray.Dataset>
Dimensions:        (time: 2160)
Coordinates:
  * time           (time) datetime64[ns] 2012-02-01 2012-02-01T01:00:00 ...
    number         int64 0
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 1000
    latitude       float64 30.0
    longitude      float64 15.0
    valid_time     (time) datetime64[ns] 2012-02-01 2012-02-01T01:00:00 ...
    surface        int64 0
Data variables:
    q              (time) float32 0.00534194 0.00553718 0.0056997 0.00581151 ...
    r              (time) float32 70.2913 75.9101 80.5836 85.3799 88.4248 ...
    t2m            (time) float32 282.83 282.76 282.2 281.418 281.33 281.021 ...
    sp             (time) float32 98076.2 98076.0 98059.1 98057.2 98028.9 ...
    tcc            (time) float64 0.5511 0.6288 0.5274 0.4335 0.4199 0.5349 ...
    nr_nans        (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...


TypeError: len() of unsized object

In [ ]:
start, stop, lat, lon, 

In [47]:
MergeError: conflicting values for variable 'valid_time' on objects to be combined:

SyntaxError: invalid syntax (<ipython-input-47-98cdb47d82e5>, line 1)

# Make a plot containing the sum of all nans over all gridded times¨

In [29]:
files = glob.glob(os.path.join( data_m, "*2007_08*.nc" ))

In [30]:
len(files)

5

In [31]:
test = xr.open_dataset(files[0])

In [32]:
test

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 744)
Coordinates:
    number         int64 ...
  * time           (time) datetime64[ns] 2007-08-01 2007-08-01T01:00:00 ...
    step           timedelta64[ns] ...
    isobaricInhPa  int64 ...
  * latitude       (latitude) float64 50.0 49.75 49.5 49.25 49.0 48.75 48.5 ...
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 -14.25 -14.0 ...
    valid_time     (time) datetime64[ns] ...
Data variables:
    r              (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2020-02-20T19:31:32 GRIB to CDM+CF via cfgrib-0....

In [33]:
datasets = [ xr.open_dataset(fil) for fil in files  ]

In [39]:
for i in range(5):
    print(datasets[i])

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 744)
Coordinates:
    number         int64 ...
  * time           (time) datetime64[ns] 2007-08-01 2007-08-01T01:00:00 ...
    step           timedelta64[ns] ...
    isobaricInhPa  int64 ...
  * latitude       (latitude) float64 50.0 49.75 49.5 49.25 49.0 48.75 48.5 ...
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 -14.25 -14.0 ...
    valid_time     (time) datetime64[ns] ...
Data variables:
    r              (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2020-02-20T19:31:32 GRIB to CDM+CF via cfgrib-0....
<xarray.Dataset>
Dimensions:     (latitude: 81, longitude: 161, time: 744)
C

In [34]:
data = xr.merge(datasets, 'equals')

In [35]:
data

<xarray.Dataset>
Dimensions:        (latitude: 81, longitude: 161, time: 744)
Coordinates:
  * latitude       (latitude) float64 30.0 30.25 30.5 30.75 31.0 31.25 31.5 ...
  * time           (time) datetime64[ns] 2007-08-01 2007-08-01T01:00:00 ...
    number         int64 0
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  int64 1000
  * longitude      (longitude) float64 -15.0 -14.75 -14.5 -14.25 -14.0 ...
    valid_time     (time) datetime64[ns] 2007-08-01 2007-08-01T01:00:00 ...
    surface        int64 0
Data variables:
    r              (time, latitude, longitude) float32 ...
    t2m            (time, latitude, longitude) float32 ...
    q              (time, latitude, longitude) float32 ...
    sp             (time, latitude, longitude) float32 ...
    tcc            (time, latitude, longitude) float64 0.0 0.0 0.0 0.0 0.0 ...
    nr_nans        (time, latitude, longitude) float64 0.0 0.0 0.0 0.0 0.0 ...

In [36]:
dropped = data.dropna(dim='time')